In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

kaggle=False
model_path="microsoft/deberta-base"

In [2]:
path='/kaggle/input/clickbait-dataset/clickbait_data.csv' if kaggle else 'train.csv'

df=pd.read_csv(path)
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [3]:
for i in tqdm(range(len(df))):
    combined=""
    if str(df.loc[i, 'keyword']) != 'nan':
        combined += 'KEYWORD: ' + str(df.loc[i, 'keyword']).replace('%20', ' ') + ' | '
    if str(df.loc[i, 'location']) != 'nan':
        combined += 'LOCATION: ' + str(df.loc[i, 'location']) + ' | '
    combined += 'TEXT: ' + df.loc[i, 'text']
    df.loc[i, 'combined'] = combined

# Drop all but combined and target
df = df.drop(columns=['id', 'keyword', 'location', 'text'])

#rename target to label
df = df.rename(columns={'target': 'label'})
df.sample(30)


  0%|          | 0/7613 [00:00<?, ?it/s]

,label,combined
7334,1,"KEYWORD: wildfire | LOCATION: Ashland, Oregon ..."
813,0,KEYWORD: blizzard | TEXT: Amazon Deal - wait o...
5486,0,KEYWORD: quarantine | TEXT: Yet another compan...
6596,1,KEYWORD: terrorism | TEXT: @RobPulseNews @huyo...
2886,0,KEYWORD: drought | LOCATION: At Work | TEXT: M...
6909,0,KEYWORD: traumatised | TEXT: @brookesddl I am ...
759,0,KEYWORD: blew up | LOCATION: Waterford MI | TE...
6410,1,KEYWORD: suicide bomber | TEXT: Suicide bomber...
7103,0,KEYWORD: violent storm | LOCATION: Milky Way g...
6814,0,KEYWORD: trapped | LOCATION: call me peach or ...


In [4]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [5]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_path)
model=AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=2)


for param in model.deberta.embeddings.parameters():
    param.requires_grad = False
        
for layer in model.deberta.encoder.layer[:6]:
    for param in layer.parameters():
        param.requires_grad = False


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from datasets import Dataset

train_data=Dataset.from_pandas(train_df)
valid_data=Dataset.from_pandas(valid_df)

train_ds=train_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)
valid_ds=valid_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

In [7]:
train_ds[99]

{'label': 0,
 'combined': 'KEYWORD: explode | TEXT: Learn How I Gained Access To The Secrets Of The Top Earners &amp; Used Them To Explode My Home Business Here: http://t.co/SGXP1U5OL1 Please #RT',
 '__index_level_0__': 3398,
 'input_ids': [1,
  33754,
  771,
  11200,
  35,
  25931,
  1721,
  36367,
  35,
  10900,
  1336,
  38,
  48564,
  8076,
  598,
  20,
  34702,
  1525,
  20,
  3107,
  7535,
  268,
  359,
  3914,
  131,
  30374,
  28595,
  598,
  16161,
  4636,
  1308,
  2193,
  2090,
  1398,
  35,
  2054,
  640,
  90,
  4,
  876,
  73,
  33020,
  29269,
  134,
  791,
  245,
  3384,
  134,
  3401,
  849,
  13963,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [8]:
from transformers import Trainer, TrainingArguments



def accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=3,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=tokenizer,
    compute_metrics=accuracy
)

In [9]:
trainer.train()

c:\Users\Bernd\anaconda3\envs\mytorch\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2286 [00:00<?, ?it/s]

You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.6867, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.6967, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.03}
{'loss': 0.6941, 'learning_rate': 3e-06, 'epoch': 0.04}
{'loss': 0.6909, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6968, 'learning_rate': 5e-06, 'epoch': 0.07}
{'loss': 0.6884, 'learning_rate': 6e-06, 'epoch': 0.08}
{'loss': 0.6737, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.09}
{'loss': 0.6911, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.1}
{'loss': 0.6879, 'learning_rate': 9e-06, 'epoch': 0.12}
{'loss': 0.6898, 'learning_rate': 1e-05, 'epoch': 0.13}


  0%|          | 0/191 [00:00<?, ?it/s]

...

  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.4711506962776184, 'eval_accuracy': 0.8365068942875903, 'eval_runtime': 2.6395, 'eval_samples_per_second': 577.003, 'eval_steps_per_second': 72.362, 'epoch': 2.76}
{'loss': 0.2897, 'learning_rate': 4.927211646136618e-06, 'epoch': 2.77}
{'loss': 0.2613, 'learning_rate': 4.647256438969766e-06, 'epoch': 2.78}
{'loss': 0.1884, 'learning_rate': 4.3673012318029114e-06, 'epoch': 2.8}
{'loss': 0.3324, 'learning_rate': 4.087346024636059e-06, 'epoch': 2.81}
{'loss': 0.3748, 'learning_rate': 3.8073908174692052e-06, 'epoch': 2.82}
{'loss': 0.4369, 'learning_rate': 3.5274356103023515e-06, 'epoch': 2.83}
{'loss': 0.3165, 'learning_rate': 3.2474804031354986e-06, 'epoch': 2.85}
{'loss': 0.3057, 'learning_rate': 2.9675251959686453e-06, 'epoch': 2.86}
{'loss': 0.2918, 'learning_rate': 2.6875699888017915e-06, 'epoch': 2.87}
{'loss': 0.2466, 'learning_rate': 2.4076147816349386e-06, 'epoch': 2.89}


  0%|          | 0/191 [00:00<?, ?it/s]

{'eval_loss': 0.4593885838985443, 'eval_accuracy': 0.8443860801050558, 'eval_runtime': 2.5544, 'eval_samples_per_second': 596.236, 'eval_steps_per_second': 74.774, 'epoch': 2.89}
{'loss': 0.442, 'learning_rate': 2.1276595744680853e-06, 'epoch': 2.9}
{'loss': 0.3417, 'learning_rate': 1.847704367301232e-06, 'epoch': 2.91}
{'loss': 0.3646, 'learning_rate': 1.5677491601343787e-06, 'epoch': 2.93}
{'loss': 0.4191, 'learning_rate': 1.2877939529675251e-06, 'epoch': 2.94}
{'loss': 0.3305, 'learning_rate': 1.0078387458006718e-06, 'epoch': 2.95}
{'loss': 0.2295, 'learning_rate': 7.278835386338187e-07, 'epoch': 2.97}
{'loss': 0.2297, 'learning_rate': 4.479283314669653e-07, 'epoch': 2.98}
{'loss': 0.369, 'learning_rate': 1.6797312430011197e-07, 'epoch': 2.99}
{'train_runtime': 155.5912, 'train_samples_per_second': 117.423, 'train_steps_per_second': 14.692, 'train_loss': 0.4024325771907496, 'epoch': 3.0}


TrainOutput(global_step=2286, training_loss=0.4024325771907496, metrics={'train_runtime': 155.5912, 'train_samples_per_second': 117.423, 'train_steps_per_second': 14.692, 'train_loss': 0.4024325771907496, 'epoch': 3.0})

In [10]:
df_test=pd.read_csv('test.csv')
df_test['combined'] = 'KEYWORD: ' + df_test['keyword'].astype(str) + ' | LOCATION: ' + df_test['location'].astype(str) + ' | TEXT: ' + df_test['text'].astype(str)
df_test = df_test.drop(columns=['id', 'keyword', 'location', 'text'])
df_test = df_test.rename(columns={'target': 'label'})


test_data=Dataset.from_pandas(df_test)

test_ds=test_data.map(lambda e: tokenizer(e['combined'], padding='max_length', truncation=True, max_length=100), batched=True)

model.eval()
preds=[]

preds=trainer.predict(test_ds)
preds=preds.predictions.argmax(-1)
df_submission_test_data=pd.read_csv('test.csv')
df_submission_test_data["target"]=preds
df_submission_test_data[["id", "target"]].to_csv("submission.csv", index=False)


Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

  0%|          | 0/408 [00:00<?, ?it/s]

In [ ]:
#83,5